SQLite Database Initialization
This section initializes the SQLite database by creating necessary tables such as Items, Auctions, and ActionEvents. It ensures the existence of the required tables and sets up the database schema.

File and Configuration Setup
Additionally, it creates other essential files and configurations needed for subsequent operations.

In [1]:
import os
import sys
import sqlite3
import json
import pandas as pd
import mysql.connector
import pickle

from datetime import datetime
from tqdm import tqdm
from pathlib import Path

wd = Path(os.path.dirname(os.path.abspath("__file__"))).parent.resolve()
sys.path.append(str(wd))

from data.transformers import add_features, transform_data

/tmp/ipykernel_3280/2294088860.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
db_path = 'auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Items (
        item_id INT PRIMARY KEY,
        item_name TEXT,
        quality TEXT,
        item_level INT,
        required_level INT,
        item_class TEXT,
        item_subclass TEXT,
        purchase_price_gold INT,
        purchase_price_silver INT,
        sell_price_gold INT,
        sell_price_silver INT,
        max_count INT,
        is_stackable INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS Auctions (
        auction_id INT PRIMARY KEY,
        bid INT,
        buyout INT,
        quantity INT,
        time_left TEXT,
        item_id INT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS ActionEvents (
        auction_id INT,
        record DATETIME,
        PRIMARY KEY (auction_id, record),
        FOREIGN KEY (auction_id) REFERENCES Auctions(auction_id)
    )
''')

conn.commit()
conn.close()

In [3]:
file_info = {}
data_dir = 'sample/'

for root, dirs, files in os.walk(data_dir):
    for filename in tqdm(files):
        filepath = os.path.join(root, filename)
        date = datetime.strptime(filename.split('.')[0], '%Y%m%dT%H')

        file_info[filepath] = date

file_info = {k: v for k, v in sorted(file_info.items(), key=lambda item: item[1])}
filenames = list(file_info.keys())

0it [00:00, ?it/s]
100%|██████████| 24/24 [00:00<00:00, 63832.15it/s]


MySQL Items Data Retrieval
In this part, the script retrieves data from a MySQL database. It reads the MySQL database configuration from a JSON file, establishes a connection, and fetches data from the Items table. The retrieved data is then loaded into a Pandas DataFrame.

SQLite Database Update
After retrieving the data, the script connects to the SQLite database, deletes all existing records from the Items table, and appends the newly fetched data.

In [4]:
db_path = 'auction.db'
data_dir = 'sample/'
db = sqlite3.connect(db_path)
cursor = db.cursor()

for i, filepath in tqdm(enumerate(filenames)):
    try:
        data = json.load(open(filepath, "r"))
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error reading file {filepath}: {e}")
        continue

    filename = filepath.split('/')[-1]
    auction_record = datetime.strptime(filename[:-5], "%Y%m%dT%H")

    if i == 0:
        auction_ids = []
        auctions_data = []

        for auction in data["auctions"]:
            if auction["id"] not in auction_ids:
                auctions_data.append((auction["id"], auction["bid"], auction["buyout"], auction["quantity"], auction["time_left"], auction["item"]["id"]))
                auction_ids.append(auction["id"])

        try:
            cursor.executemany("""
                INSERT INTO Auctions (auction_id, bid, buyout, quantity, time_left, item_id)
                VALUES (?, ?, ?, ?, ?, ?)
            """, auctions_data)
            db.commit()
        except sqlite3.Error as err:
            db.rollback()
            print(f"Error inserting auction data for file {filepath} in Auctions: {err}")

    action_events_data = []
    for auction in data["auctions"]:
        action_events_data.append((auction["id"], auction_record.strftime('%Y-%m-%d %H:%M:%S')))
            
    try:
        cursor.executemany("""
            INSERT OR REPLACE INTO ActionEvents (auction_id, record)
            VALUES (?, ?)
        """, action_events_data)
        db.commit()
    except sqlite3.Error as err:
        db.rollback()
        print(f"Error inserting auction events for file {filepath} in ActionEvents: {err}")

cursor.close()
db.close()

48it [01:44,  2.18s/it]


Auction Data and Items Storage
This section processes JSON files containing auction data. It iterates through the files, extracts relevant information, and inserts it into the Auctions and ActionEvents tables of the SQLite database.

Data Import from Pandas DataFrame
Moreover, it initializes a connection to the SQLite database and imports data from a Pandas DataFrame into the Items table. This DataFrame is generated from MySQL database retrieval, ensuring the SQLite Items table is up-to-date.

In [5]:
with open('../data/config.json') as f:
    config = json.load(f)

db_path = 'auction.db'
query = "SELECT * FROM Items"

def import_items():
    try:
        mysql_db = mysql.connector.connect(**config['database'])
    except mysql.connector.Error as err:
        print(err)
        return

    cursor = mysql_db.cursor()

    items = cursor.execute(query)
    items = cursor.fetchall()

    mysql_db.close()
    cursor.close()

    try:
        db = sqlite3.connect(db_path)
        print("Connected to SQLite")
    except sqlite3.Error as err:
        return

    cursor = db.cursor()
    cursor.executemany("""
        INSERT OR REPLACE INTO Items (item_id, item_name, quality, item_level, required_level, item_class, item_subclass, purchase_price_gold, purchase_price_silver, sell_price_gold, sell_price_silver, max_count, is_stackable)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, items)

    db.commit()

    cursor.close()
    db.close()

    print("Inserted items into SQLite: " + str(len(items)))

import_items()

Connected to SQLite
Inserted items into SQLite: 10396


# Read auction.db
It is responsible for connecting to an SQLite database, executing a SQL query involving multiple tables, and retrieving the results. These results are stored in the variable

In [21]:
db_path = 'auction.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

query = """
    SELECT
        a.auction_id,
        a.bid / 10000.0 AS bid_in_gold,
        a.buyout / 10000.0 AS buyout_in_gold,
        (a.buyout / 10000.0) / a.quantity AS unit_price,
        a.quantity,
        a.time_left,
        a.item_id,
        i.item_name,
        i.quality,
        i.item_class,
        i.item_subclass,
        i.is_stackable,
        i.purchase_price_gold,
        i.required_level,
        i.item_level,
        i.sell_price_gold,
        MIN(ae.record) AS first_appearance_timestamp,
        strftime('%Y', MIN(ae.record)) AS first_appearance_year,
        strftime('%m', MIN(ae.record)) AS first_appearance_month,
        strftime('%d', MIN(ae.record)) AS first_appearance_day,
        strftime('%H', MIN(ae.record)) AS first_appearance_hour,
        COUNT(*) AS hours_on_sale
    FROM Auctions a
    JOIN ActionEvents ae ON a.auction_id = ae.auction_id
    JOIN Items i ON i.item_id = a.item_id
    WHERE A.time_left <> 'SHORT'
    GROUP BY a.auction_id
"""

cursor.execute(query)
results = cursor.fetchall()

conn.close()

df = pd.DataFrame(results, columns=[i[0] for i in cursor.description])
df.head(10)

,auction_id,bid_in_gold,buyout_in_gold,unit_price,quantity,time_left,item_id,item_name,quality,item_class,...,purchase_price_gold,required_level,item_level,sell_price_gold,first_appearance_timestamp,first_appearance_year,first_appearance_month,first_appearance_day,first_appearance_hour,hours_on_sale
0,1017838219,2,2,2,1,MEDIUM,41108,Glyph of Divinity,Common,Glyph,...,0,15,15,0,2024-02-23 00:00:00,2024,02,23,00,3
1,1017838298,2,2,2,1,MEDIUM,42402,Glyph of Inner Fire,Common,Glyph,...,0,15,17,0,2024-02-23 00:00:00,2024,02,23,00,3
2,1017838396,1,1,1,1,MEDIUM,42964,Glyph of Garrote,Common,Glyph,...,0,15,19,0,2024-02-23 00:00:00,2024,02,23,00,3
3,1017838401,1,1,1,1,MEDIUM,42964,Glyph of Garrote,Common,Glyph,...,0,15,19,0,2024-02-23 00:00:00,2024,02,23,00,3
4,1017839984,3,0,0,1,MEDIUM,15323,Percussion Shotgun,Uncommon,Weapon,...,8,45,50,1,2024-02-23 00:00:00,2024,02,23,00,3
5,1017840012,7,0,0,1,MEDIUM,15264,Backbreaker,Uncommon,Weapon,...,20,51,56,4,2024-02-23 00:00:00,2024,02,23,00,3
6,1017840126,10,0,0,1,MEDIUM,25298,Combustion Dagger,Uncommon,Weapon,...,29,59,87,5,2024-02-23 00:00:00,2024,02,23,00,3
7,1017840206,110,110,110,1,MEDIUM,8345,Wolfshead Helm,Rare,Armor,...,3,40,45,0,2024-02-23 00:00:00,2024,02,23,00,3
8,1017840284,1,0,0,1,MEDIUM,10058,Duskwoven Sandals,Uncommon,Armor,...,3,45,50,0,2024-02-23 00:00:00,2024,02,23,00,3
9,1017840329,46,48,48,1,MEDIUM,34330,Heavy Knothide Armor Kit,Common,Consumable,...,3,60,70,0,2024-02-23 00:00:00,2024,02,23,00,3


In [22]:
# increase buyout and bid 10 times for the first 10 records
df.loc[:9, 'buyout_in_gold'] = df.loc[:9, 'buyout_in_gold'] * 30
df.loc[:9, 'bid_in_gold'] = df.loc[:9, 'bid_in_gold'] * 30
df['unit_price'] = df['buyout_in_gold'] / df['quantity']


In [23]:
df = add_features(df)

In [24]:
print(f'Dataframe shape before duplicates {df.shape}')

df = df.drop_duplicates(subset=['bid_in_gold', 'buyout_in_gold', 'quantity', 'time_left', 'first_appearance_timestamp', 'item_id'])
df = df[(df['time_left'] != 2)]

print(f'Dataframe shape after duplicates {df.shape}')

Dataframe shape before duplicates (102713, 39)
Dataframe shape after duplicates (39030, 39)


In [25]:
X, y = transform_data(df)

df.head(5)

,auction_id,bid_in_gold,buyout_in_gold,unit_price,quantity,time_left,item_id,item_name,quality,item_class,...,std_competitor_price,competitor_count,lowest_competitor_price,top_competitor_price,relative_price_difference,relative_avg_price_difference,relative_buyout_difference,relative_bid_difference,relative_price_to_lowest_competitor,relative_price_to_top_competitor
0,1017838219,57,60,60,1,12,41108,Glyph of Divinity,Common,Glyph,...,11,30,2,2,29,5,29,29,29,28
1,1017838298,46,48,48,1,12,42402,Glyph of Inner Fire,Common,Glyph,...,11,17,2,5,29,4,29,29,29,8
2,1017838396,36,37,37,1,12,42964,Glyph of Garrote,Common,Glyph,...,12,17,1,4,29,3,29,29,29,9
4,1017839984,80,0,0,1,12,15323,Percussion Shotgun,Uncommon,Weapon,...,8,2,2,9,-1,-1,-1,1,-1,-1
5,1017840012,217,0,0,1,12,15264,Backbreaker,Uncommon,Weapon,...,60,2,13,68,-1,-1,-1,0,-1,-1


In [5]:
df.columns

Index(['auction_id', 'bid_in_gold', 'buyout_in_gold', 'unit_price', 'quantity',
       'time_left', 'item_id', 'item_name', 'quality', 'item_class',
       'item_subclass', 'is_stackable', 'purchase_price_gold',
       'required_level', 'item_level', 'sell_price_gold',
       'first_appearance_timestamp', 'first_appearance_year',
       'first_appearance_month', 'first_appearance_day',
       'first_appearance_hour', 'hours_on_sale', 'median_buyout_price',
       'median_bid_price', 'median_unit_price', 'rank_buyout_price',
       'rank_bid_price', 'rank_unit_price', 'avg_competitor_price',
       'std_competitor_price', 'competitor_count', 'lowest_competitor_price',
       'top_competitor_price', 'relative_price_difference',
       'relative_avg_price_difference', 'relative_buyout_difference',
       'relative_bid_difference', 'relative_price_to_lowest_competitor',
       'relative_price_to_top_competitor'],
      dtype='object')

loads a trained model from a file, makes predictions on a data set, calculates the RMSE and displays the result

In [6]:
pd.options.display.float_format = '{:.0f}'.format

with open('models/forest_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

,item_name,item_class,unit_price,bid_in_gold,buyout_in_gold,time_left,hours_on_sale,prediction
0,Glyph of Divinity,Glyph,2,2,2,12,3,21
1,Glyph of Inner Fire,Glyph,2,2,2,12,3,24
2,Glyph of Garrote,Glyph,1,1,1,12,3,24
3,Glyph of Garrote,Glyph,1,1,1,12,3,24
4,Percussion Shotgun,Weapon,0,3,0,12,3,29


In [26]:
predictions = model.predict(X)
df['prediction'] = predictions
df[['item_name', 'item_class', 'unit_price', 'bid_in_gold', 'buyout_in_gold', 'time_left', 'hours_on_sale', 'prediction']].head(5)

,item_name,item_class,unit_price,bid_in_gold,buyout_in_gold,time_left,hours_on_sale,prediction
0,Glyph of Divinity,Glyph,60,57,60,12,3,24
1,Glyph of Inner Fire,Glyph,48,46,48,12,3,23
2,Glyph of Garrote,Glyph,37,36,37,12,3,22
4,Percussion Shotgun,Weapon,0,80,0,12,3,23
5,Backbreaker,Weapon,0,217,0,12,3,29


In [27]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y, predictions)

print("MAE:", mae)

MAE: 9.899862817571844


In [11]:
df[['item_name', 'item_class', 'unit_price', 'bid_in_gold', 'buyout_in_gold', 'time_left', 'first_appearance_timestamp', 'median_buyout_price',
       'median_bid_price', 'median_unit_price', 'rank_buyout_price',
       'rank_bid_price', 'rank_unit_price', 'avg_competitor_price',
       'std_competitor_price', 'competitor_count', 'lowest_competitor_price',
       'top_competitor_price', 'relative_price_difference',
       'relative_avg_price_difference', 'relative_buyout_difference',
       'relative_bid_difference', 'relative_price_to_lowest_competitor',
       'relative_price_to_top_competitor', 'hours_on_sale', 'prediction']].to_csv('predictions.csv', index=False)